In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [6]:
data = pd.read_csv("Hip_Hop _ Vocab_Chart-Full.csv")

In [7]:
data.rename(index=str, columns={"rapper": "artist"}, inplace=True)
data.drop(columns=['rapper_clean', 'notes', 'color'], inplace=True)
data.drop(columns=['words'], inplace=True)
data.rename(index=str, columns={"recalc": "words"}, inplace=True)

In [8]:
# anova test to see atleast one mean is statistically different from the rest, given low p-val, yes.
stats.f_oneway(data['words'][data['era'] == '1980s'],
             data['words'][data['era'] == '1990s'],
             data['words'][data['era'] == '2000s'],
             data['words'][data['era'] == '2010s'])

F_onewayResult(statistic=12.72696641437611, pvalue=1.7637948016674944e-07)

In [9]:
# fishers LSD to find out which means are stat. diff.
# drop from 2000s to 2010s (and 1990s to 2010s) is stat. sig
a = stats.ttest_ind(data['words'][data['era'] == '1980s'], data['words'][data['era'] == '1990s'])
b = stats.ttest_ind(data['words'][data['era'] == '1980s'], data['words'][data['era'] == '2000s'])
c = stats.ttest_ind(data['words'][data['era'] == '1980s'], data['words'][data['era'] == '2010s'])
d = stats.ttest_ind(data['words'][data['era'] == '1990s'], data['words'][data['era'] == '2000s'])
e = stats.ttest_ind(data['words'][data['era'] == '1990s'], data['words'][data['era'] == '2010s'])
f = stats.ttest_ind(data['words'][data['era'] == '2000s'], data['words'][data['era'] == '2010s'])
a,b,c,d,e,f

(Ttest_indResult(statistic=-1.1542042683286322, pvalue=0.25390347566862714),
 Ttest_indResult(statistic=-1.115576134045436, pvalue=0.26936798984985877),
 Ttest_indResult(statistic=1.860506106507783, pvalue=0.06748256111528092),
 Ttest_indResult(statistic=-0.4620185970644447, pvalue=0.6451586998726611),
 Ttest_indResult(statistic=5.5800446340959935, pvalue=2.0987336234645824e-07),
 Ttest_indResult(statistic=5.358893197884523, pvalue=4.989772891200519e-07))

In [10]:
# alternatively can do one_way anova tests but doing them pairwise gives same res as doing t tests 
# (which are by def supposed to be done between 2 groups)
data_anova = data[['era','words']].groupby(['era'])
anova_res_1 = stats.f_oneway(data_anova.get_group('2000s')['words'],data_anova.get_group('2010s')['words'])
anova_res_2 = stats.f_oneway(data_anova.get_group('1980s')['words'],data_anova.get_group('1990s')['words'])
anova_res_3 = stats.f_oneway(data_anova.get_group('1990s')['words'],data_anova.get_group('2000s')['words'])
anova_res_1, anova_res_2,anova_res_3

(F_onewayResult(statistic=28.71773630633302, pvalue=4.989772891200487e-07),
 F_onewayResult(statistic=1.3321874930280369, pvalue=0.25390347566862653),
 F_onewayResult(statistic=0.21346118403339606, pvalue=0.6451586998726657))

In [11]:
# need to check whether conditions meet for doing anova test
# levene test checks for homogeneity of var between groups
# since not stat sig (p-val greater that 0.05), can assume this condition
# need to be more sure of this and check all conditions before doing anova tests
stats.levene(data['words'][data['era'] == '1990s'],
             data['words'][data['era'] == '2010s'])

LeveneResult(statistic=0.3044665045638414, pvalue=0.5823391897091525)

In [14]:
# this is just to know who the min max artists are 
max_idx = data[['era','words','artist']].groupby('era')['words'].idxmax()
min_idx = data[['era','words','artist']].groupby('era')['words'].idxmin()
max_words = data.loc[max_idx,][['era','words','artist']]
min_words = data.loc[min_idx,][['era','words','artist']]